## Bike-rental volume prediction

The purpose of this notebook is to provide an illustration of how the freshq algorithm can be implemented to aid in the forecasting the value of a target variable without the implementation of data rolling and for data which contains inconsistent evenly spaced data points.

## Loading library scripts and data

In the following cell the functions related to the FRESH library are loaded in the 1st line while preprocessing functions used within the notebook are loaded from the folder mlutils

In [1]:
\l ../../ml.q
.ml.loadfile`:init.q
\l graphics.q
\c 15 100

Load in data pertaining to a bike sharing scheme in Washington DC between 2011-2012 (This data was obtained from kaggle at: https://www.kaggle.com/marklvl/bike-sharing-dataset).

Our goal is to use the hourly data from the current day to predict if the last hour of the next day will be busier or quieter than the last hour of today.

This use case while slightly contrived and chosen to show the operation of this this algorithm, it could potentially useful in allowing an employer to anticipate if extra staff are required for the next business day, which could save on costs.

In [2]:
tabinit:("IDIIIIIIIIFFFFIII";enlist ",") 0:`:SampleDatasets/bike_rental.csv

/drop columns which contain zero variance as these would not be useful for use in a machine learning algorithm.
show tabinit:flip (where 0=var each flip tabinit) _ flip tabinit

instant dteday     season yr mnth hr holiday weekday workingday weathersit temp atemp  hum  winds..
-------------------------------------------------------------------------------------------------..
1       2011.01.01 1      0  1    0  0       6       0          1          0.24 0.2879 0.81 0    ..
2       2011.01.01 1      0  1    1  0       6       0          1          0.22 0.2727 0.8  0    ..
3       2011.01.01 1      0  1    2  0       6       0          1          0.22 0.2727 0.8  0    ..
4       2011.01.01 1      0  1    3  0       6       0          1          0.24 0.2879 0.75 0    ..
5       2011.01.01 1      0  1    4  0       6       0          1          0.24 0.2879 0.75 0    ..
6       2011.01.01 1      0  1    5  0       6       0          2          0.24 0.2576 0.75 0.089..
7       2011.01.01 1      0  1    6  0       6       0          1          0.22 0.2727 0.8  0    ..
8       2011.01.01 1      0  1    7  0       6       0          1          0.2  0.2576 0.86 0    ..


---

## Set the target values for forecasting

Next we set the target which will be estimated from the dataset. In this case we are attempting to predict if the number of bike rentals in the last hour of business will be higher or lower tomorrow than it was today.

In [3]:
lasthour:value exec last cnt by dteday from tabinit
targets:lasthour>prev lasthour

## Naive solution

Under the naive solution i.e choosing that the number of rentals price will only ever increase we would be correct ~56% of the time, as such if our procedure and algorithm have provided us with new insights into the data our predictions will be >56% on a consistent basis. 

In [4]:
update pcnt:100*num%sum num from select num:count i by target from ([]target:targets)

target| num pcnt    
------| ------------
0     | 319 43.63885
1     | 412 56.36115


## Assign extracted features and complete extraction

In [5]:
/ in this example we look only at features of the data alone with no parameters

show singleinputfeatures:.ml.fresh.getsingleinputfeatures[]

\t tabraw:.ml.fresh.createfeatures[tabinit;`dteday;2_ cols tabinit;singleinputfeatures]
show tabraw
-1"The number of rows in the data is: ",string count tabraw;
-1"The number of targets which are to be predicted is: ",string count lasthour;

absenergy     | {x wsum x}
abssumchange  | {sum abs 1_deltas x}
count         | {count x}
countabovemean| {sum x>avg x}
countbelowmean| {sum x<avg x}
firstmax      | {(x?max x)%count x}
firstmin      | {(x?min x)%count x}
hasdup        | {count[x]<>count distinct x}
hasdupmax     | {1<sum x=max x}
hasdupmin     | {1<sum x=min x}
kurtosis      | {((n-1)%(n-2)*n-3)*((n+1)*n*sum[k2*k2]%
 	     s*s:sum k2:k*k:x-avg x)+3*1-n:coun..
lastmax       | {(last where x=max x)%count x}
..


1110


dteday    | absenergy_season absenergy_yr absenergy_mnth absenergy_hr absenergy_holiday absenergy..
----------| -------------------------------------------------------------------------------------..
2011.01.01| 24               0            24             4324         0                 864      ..
2011.01.02| 23               0            23             4299         0                 0        ..
2011.01.03| 22               0            22             4311         0                 22       ..
2011.01.04| 23               0            23             4315         0                 92       ..
2011.01.05| 23               0            23             4315         0                 207      ..
2011.01.06| 23               0            23             4315         0                 368      ..
2011.01.07| 23               0            23             4315         0                 575      ..
2011.01.08| 24               0            24             4324         0                 864      ..


## Complete feature significance tests

Upon completion of the feature extraction algorithm the importance of each of the features can be determined through the statistical tests contained in the .fresh.significantfeatures function. This will reduce the number of features used by the machine learning algorithm in making its prediction.

In [6]:
show tabreduced:key[tabraw]!(.ml.fresh.significantfeatures[t;targets])#t:value tabraw
-1 "The number of columns in the initial dataset is: ",string count cols tabinit;
-1 "The number of columns in the unfiltered dataset is: ",string count cols tabraw;
-1 "The number of columns in the filtered dataset is: ",string count cols tabreduced;

dteday    | meanchange_cnt meanchange_registered max_workingday mean_workingday med_workingday mi..
----------| -------------------------------------------------------------------------------------..
2011.01.01| 1              0.4782609             0              0               0              0 ..
2011.01.02| -0.4090909     -0.2272727            0              0               0              0 ..
2011.01.03| 0.3333333      0.2857143             1              1               1              1 ..
2011.01.04| 0.2727273      0.1818182             1              1               1              1 ..
2011.01.05| 0.5909091      0.5454545             1              1               1              1 ..
2011.01.06| 0.5454545      0.3636364             1              1               1              1 ..
2011.01.07| -0.09090909    0.04545455            1              1               1              1 ..
2011.01.08| -0.1304348     -0.08695652           0              0               0              0 ..


### Data formatting

The data must now be converted to a matrix from a table in order to allow it to be passed to a machine learning algorithm for training.

In [7]:
mattab:{flip value flip x}
fitvalsfilter:0^mattab[value tabreduced]

---

## Training the model

Below we initialize the model with a random forest classifier with 200 estimators, this allows for an estimate of the quality of the data to be made following the completion of feature extraction. The addition of ``` `verbose pykw 1 ``` to the definition of the Random Forest Classifier allows for the time required for the training of the classifier to be displayed if desired.

In [8]:
clf:.p.import[`sklearn.ensemble][`:RandomForestClassifier][`n_estimators pykw 200;`random_state pykw 42]
classreport:.p.import[`sklearn.metrics]`:classification_report

### Fit data to the models and make predictions

In [9]:
seed:"i"$.z.t
fitvalsfilter:.ml.util.infreplace[fitvalsfilter]
dict1:.ml.util.traintestsplitseed[fitvalsfilter;targets;0.2;seed];
clf[`:fit][dict1[`xtrain];dict1[`ytrain]]`;
pred1:clf[`:predict][dict1[`xtest]]`

## Results

Given that predictions of the classifications given to the whether or not the last hour of work tomorrow will be busier than today are being made .

In [10]:
print classreport[dict1[`ytest];pred1]`
-1"The number of misclassifications in the filtered dataset is: ",string sum dict1[`ytest]<>pred1;
-1"The accuracy in the filtered dataset is: ",string .ml.accuracy[dict1[`ytest];pred1];
-1"_______________________________________________________________";

              precision    recall  f1-score   support

       False       0.76      0.71      0.74        63
        True       0.80      0.83      0.81        84

   micro avg       0.78      0.78      0.78       147
   macro avg       0.78      0.77      0.78       147
weighted avg       0.78      0.78      0.78       147

The number of misclassifications in the filtered dataset is: 32
The accuracy in the filtered dataset is: 0.7823129
_______________________________________________________________


0| 45 14
1| 18 70


In [11]:
show cnfM:.ml.cfm[dict1[`ytest];pred1]
.ml.displayCM[value cnfM;`Low`High;"Test Set Confusion Matrix";()]

## Conclusions

In conclusion as can be seen from the above results the use of a random forest classifier was useful in allowing for the classification of next day last hour business vs previous day, to be more consistent than simply choosing one case to always be true.

It should also be noted that the quality of results garnered from the running of the classifications depends on the location that the data is split. This is an inherent problem in predicting on human behaviour influenced systems as it is difficult to predict on outlying cases which have a non repeating signature.

---